In [3]:
import pandas as pd
import folium as folium

from folium import plugins
from collections import defaultdict

In [4]:
ufo = pd.read_csv('./ufo-scrubbed-geocoded-time-standardized.csv', header=None)
ufo = ufo.drop(columns=[6, 8, 9, 10])
ufo = ufo.rename(columns={0:'occurrence', 1:'city', 2:'state', 3:'country', 4:'shape', 
                            5:'time_seconds', 7:'desc'})
ufo = ufo[ufo.country == 'us']
ufo = ufo[ufo.state != 'hi']
ufo = ufo[ufo.state != 'ak'].reset_index()
ufo = ufo.drop(columns=['index', 'country'])
ufo

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,occurrence,city,state,shape,time_seconds,desc
0,10/10/1949 20:30,san marcos,tx,cylinder,2700,This event took place in early fall around 194...
1,10/10/1956 21:00,edna,tx,circle,20,My older brother and twin sister were leaving ...
2,10/10/1961 19:00,bristol,tn,sphere,300,My father is now 89 my brother 52 the girl wit...
3,10/10/1965 23:45,norwalk,ct,disk,1200,A bright orange color changing to reddish colo...
4,10/10/1966 20:00,pell city,al,disk,180,Strobe Lighted disk shape object observed clos...
...,...,...,...,...,...,...
64528,9/9/2013 21:15,nashville,tn,light,600,Round from the distance/slowly changing colors...
64529,9/9/2013 22:00,boise,id,circle,1200,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...
64530,9/9/2013 22:00,napa,ca,other,1200,Napa UFO&#44
64531,9/9/2013 22:20,vienna,va,circle,5,Saw a five gold lit cicular craft moving fastl...


In [5]:
cities = pd.read_csv('./uscities.csv')
cities = cities[['city', 'state_id', 'lat', 'lng']]
cities['city'] = cities['city'].str.lower()
cities['state_id'] = cities['state_id'].str.lower()
cities = cities.rename(columns={'state_id':'state'})
cities

,city,state,lat,lng
0,south creek,wa,46.9994,-122.3921
1,roslyn,wa,47.2507,-121.0989
2,sprague,wa,47.3048,-117.9713
3,gig harbor,wa,47.3352,-122.5968
4,lake cassidy,wa,48.0639,-122.0920
...,...,...,...,...
28884,montrose,sd,43.7002,-97.1843
28885,spearfish,sd,44.4912,-103.8167
28886,emery,sd,43.6020,-97.6195
28887,watertown,sd,44.9094,-97.1532


In [6]:
data = pd.merge(ufo, cities, on=['city','state'])
data

,occurrence,city,state,shape,time_seconds,desc,lat,lng
0,10/10/1949 20:30,san marcos,tx,cylinder,2700,This event took place in early fall around 194...,29.8736,-97.9380
1,10/2/2005 09:15,san marcos,tx,diamond,5400,Very bright light was seen over South direction,29.8736,-97.9380
2,12/10/1965 21:00,san marcos,tx,light,60,What happened on that buss in 1965&#44 in Texas,29.8736,-97.9380
3,1/24/2014 19:23,san marcos,tx,disk,720,I was heading toward downtown from my house on...,29.8736,-97.9380
4,1/30/2009 20:04,san marcos,tx,triangle,1500,object flew over me&#44 in triangle shape with...,29.8736,-97.9380
...,...,...,...,...,...,...,...,...
55304,9/9/2000 03:30,gardiner,mt,light,120,Yankee Stadium in the middle of nowhere,45.0570,-110.7421
55305,9/9/2000 21:30,white oak,ms,fireball,20,fireball with small round object leaving it af...,34.6438,-90.3495
55306,9/9/2009 20:45,mountain,wi,light,120,obj with comet tail followed by another object...,45.1864,-88.4681
55307,9/9/2010 22:00,lemitar,nm,fireball,300,orange lights in the southeastern sky in New M...,34.1551,-106.9122


In [7]:
counts = defaultdict(lambda:[0, 0, 0])

for spot in data.values:
    key = f"{spot[1].title()}, {spot[2].upper()}"
    if key in counts.keys():
        counts[key][2] += 1
    else:
        counts[key][0] += spot[6]
        counts[key][1] += spot[7]
        counts[key][2] += 1
        
# {'City, STATE': [lat, lng, count]}
# counts

In [8]:
m = folium.Map()
for loc, count in counts.items():
    folium.Marker(location=[count[0], count[1]], popup=f"{loc}: {count[2]}").add_to(m)

In [9]:
len(counts)

9819

In [10]:
coords = data[['lat', 'lng']]
coords

,lat,lng
0,29.8736,-97.9380
1,29.8736,-97.9380
2,29.8736,-97.9380
3,29.8736,-97.9380
4,29.8736,-97.9380
...,...,...
55304,45.0570,-110.7421
55305,34.6438,-90.3495
55306,45.1864,-88.4681
55307,34.1551,-106.9122


In [11]:
print(min(data[['occurrence']].values))
print(max(data[['occurrence']].values))

['1/1/1944 12:00']
['9/9/2013 23:00']


In [12]:
time_dict = defaultdict(list)
for i in range(1940, 2011, 10):
    time_dict[f"{i}s"]
    
for spot in data.values:
    year = int(spot[0].split(' ')[0].split('/')[2])
    if year < 1950:
        time_dict['1940s'].append([spot[6], spot[7]])
    elif year < 1960:
        time_dict['1950s'].append([spot[6], spot[7]])
    elif year < 1970:
        time_dict['1960s'].append([spot[6], spot[7]])
    elif year < 1980:
        time_dict['1970s'].append([spot[6], spot[7]])
    elif year < 1990:
        time_dict['1980s'].append([spot[6], spot[7]])
    elif year < 2000:
        time_dict['1990s'].append([spot[6], spot[7]])
    elif year < 2010:
        time_dict['2000s'].append([spot[6], spot[7]])
    else:
        time_dict['2010s'].append([spot[6], spot[7]])
        
time_array = [array for array in time_dict.values()]

In [70]:
hm = folium.Map(location=[38,-95], zoom_start=4)
hm.add_child(plugins.HeatMap(time_array[7], min_opacity=0.5, radius=15,
                             gradient={1:'#EC266E', 0.8:'#FFFD78',
                                       0.7:'#73FB79', 0.55:'#73FEFF', 
                                       0.35:'#AD6AFF'}))

In [71]:
for time, coords in time_dict.items():
    hm = folium.Map(location=[38,-95], zoom_start=4)
    hm.add_child(plugins.HeatMap(coords, min_opacity=0.5, radius=15,
                                 gradient={1:'#EC266E', 0.8:'#FFFD78',
                                           0.7:'#73FB79', 0.55:'#73FEFF', 
                                           0.35:'#AD6AFF'}))
    hm.save(f'ufo-site/static/{time}.html')